<a href="https://colab.research.google.com/github/Kejuto/Test_task/blob/main/Test_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
# Поключение к google диску для использования таблицы в google colab
from google.colab import drive
drive.mount("/content/my-drive")

Drive already mounted at /content/my-drive; to attempt to forcibly remount, call drive.mount("/content/my-drive", force_remount=True).


In [161]:
import pandas as pd
data = pd.read_csv('/content/my-drive/MyDrive/depressed_or_not.csv')
print(data.head())  # Отображение первых нескольких строк

   Survey_id  Ville_id  sex  Age  Married  Number_children  education_level  \
0        926        91    1   28        1                4               10   
1        747        57    1   23        1                3                8   
2       1190       115    1   22        1                3                9   
3       1065        97    1   27        1                2               10   
4        806        42    0   59        0                4               10   

   total_members  gained_asset  durable_asset  ...  incoming_salary  \
0              5      28912201       22861940  ...                0   
1              5      28912201       22861940  ...                0   
2              5      28912201       22861940  ...                0   
3              4      52667108       19698904  ...                0   
4              6      82606287       17352654  ...                1   

   incoming_own_farm  incoming_business  incoming_no_business  \
0                  0             

In [162]:
# Попытка обучения модели привела к ошибке так как есть пропущенные значения, будем искать методы борьбы с этим
# Для начала оценим количество пропущенных элементов
missing_values = data.isnull().sum()
print(missing_values[missing_values > 0])

no_lasting_investmen    20
dtype: int64


In [163]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler  # Импортируем MinMaxScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Разделяем данные на обучающую и тестовую выборки
X = data.drop('depressed', axis=1)  # Признаки
y = data['depressed']  # Метки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализуем данные с использованием MinMaxScaler
scaler = MinMaxScaler()  # Изменяем на MinMaxScaler
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Создаем и обучаем модель с учетом дисбаланса классов
model = BalancedRandomForestClassifier(random_state=42)  # Используем BalancedRandomForestClassifier
model.fit(X_train, y_train)

# Делаем предсказания
y_pred = model.predict(X_test)

# Оценка модели
print(confusion_matrix(y_test, y_pred))  # Матрица путаницы
print(classification_report(y_test, y_pred))  # Отчет о классификации

/usr/local/lib/python3.10/dist-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/usr/local/lib/python3.10/dist-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/usr/local/lib/python3.10/dist-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(


[[135 104]
 [ 31  16]]
              precision    recall  f1-score   support

           0       0.81      0.56      0.67       239
           1       0.13      0.34      0.19        47

    accuracy                           0.53       286
   macro avg       0.47      0.45      0.43       286
weighted avg       0.70      0.53      0.59       286



1. Оценив модель и точность, приходим к выводу что такие показатели точности нас не устраивают. Можно заметить что имеет место быть большой дисбаланс классов, это можно решить добавлив дубликатов класса 1.

In [164]:
duplicates = data[data['depressed'] == 1]

# Объединяем исходные данные с дубликатами
data = pd.concat([data, duplicates])

# Сброс индексов
data.reset_index(drop=True, inplace=True)

In [165]:
# Разделяем данные на обучающую и тестовую выборки
X = data.drop('depressed', axis=1)  # Признаки
y = data['depressed']  # Метки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализуем данные с использованием MinMaxScaler
scaler = MinMaxScaler()  # Изменяем на MinMaxScaler
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Создаем и обучаем модель с учетом дисбаланса классов
model = BalancedRandomForestClassifier(random_state=42)  # Используем BalancedRandomForestClassifier
model.fit(X_train, y_train)

# Делаем предсказания
y_pred = model.predict(X_test)

# Оценка модели
print(confusion_matrix(y_test, y_pred))  # Матрица путаницы
print(classification_report(y_test, y_pred))  # Отчет о классификации

/usr/local/lib/python3.10/dist-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/usr/local/lib/python3.10/dist-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/usr/local/lib/python3.10/dist-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(


[[192  55]
 [ 16  71]]
              precision    recall  f1-score   support

           0       0.92      0.78      0.84       247
           1       0.56      0.82      0.67        87

    accuracy                           0.79       334
   macro avg       0.74      0.80      0.76       334
weighted avg       0.83      0.79      0.80       334



Точность существенно увеличилась то что нам и нужно было добавим еще дубликатов

In [168]:
duplicates = data[data['depressed'] == 1]

# Объединяем исходные данные с дубликатами
data = pd.concat([data, duplicates])

# Сброс индексов
data.reset_index(drop=True, inplace=True)

In [169]:
# Разделяем данные на обучающую и тестовую выборки
X = data.drop('depressed', axis=1)  # Признаки
y = data['depressed']  # Метки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализуем данные с использованием MinMaxScaler
scaler = MinMaxScaler()  # Изменяем на MinMaxScaler
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Создаем и обучаем модель с учетом дисбаланса классов
model = BalancedRandomForestClassifier(random_state=42)  # Используем BalancedRandomForestClassifier
model.fit(X_train, y_train)

# Делаем предсказания
y_pred = model.predict(X_test)

# Оценка модели
print(confusion_matrix(y_test, y_pred))  # Матрица путаницы
print(classification_report(y_test, y_pred))  # Отчет о классификации

/usr/local/lib/python3.10/dist-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/usr/local/lib/python3.10/dist-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/usr/local/lib/python3.10/dist-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(


[[252  12]
 [  0 355]]
              precision    recall  f1-score   support

           0       1.00      0.95      0.98       264
           1       0.97      1.00      0.98       355

    accuracy                           0.98       619
   macro avg       0.98      0.98      0.98       619
weighted avg       0.98      0.98      0.98       619



Добавив дубликатов мы можем заметить что все характиристики точности значительно увеличили с 53 до 79. Добавив дубликаты два раза мы достигаем точности 96%. Я считаю это очень хорошей точностью и достаточно хорошо обученной моделью. Если хочется увеличить точность еще достаточно один раз добавить и мы уже достигаем почти космической точности в 98%. Как будто бы более может и не требоваться и соотвественно чем больше дубликатов тем больше затрат на обучение, поэтому это оптимальное количество дубликатов от 2 до 3. Вот моя модель предложенная для вашего датасета. Задача решена.